# STA 141B Lecture 14

The class website is <https://github.com/2019-winter-ucdavis-sta141b/notes>

### Announcements

* Want additional feedback or a regrade on Assignment 3? Email Shan (<shjiang@ucdavis.edu>) and CC me (<naulle@ucdavis.edu>).

### Topics

* Databases & SQL

### Datasets

* The [Suppliers Database](http://nick-ulle.github.io/teach/suppliers.sqlite)

### References

* [W3 Schools SQL Tutorial](https://www.w3schools.com/sql/)
* [SQL Cheatsheet](http://anson.ucdavis.edu/~clarkf/sql/sql_cheatsheet.pdf)

[PDSH]: https://jakevdp.github.io/PythonDataScienceHandbook/
[ProGit]: https://git-scm.com/book/
[nlpp]: https://www.nltk.org/book/
[atap]: https://search.library.ucdavis.edu/primo-explore/fulldisplay?docid=01UCD_ALMA51320822340003126&context=L&vid=01UCD_V1&search_scope=everything_scope&tab=default_tab&lang=en_US

In [ ]:
import numpy as np
import pandas as pd

## Databases

A _database_ is a collection of data. There are several different models for how to organize data in a database; these are called _database models_. In this context, "model" refers to a design or mental model, not a statistical model.

The _relational model_ organizes data as a collection of tables. Tables have rows (also called _tuples_ or _records_) and columns (also called _attributes_). Most tables have a _key_ column that is unique for each row and _relates_ the table to other tables. The relational model is the most popular database model by far, and the one we'll focus on in this course.

There are also many different software programs for managing databases, called _database management systems_ (DBMS). Each DBMS usually has its own format for storing data on disk, independent of the database model. Some popular DBMSes are:

* [SQLite](https://www.sqlite.org/)
* [MySQL](https://www.mysql.com/)
* [Microsoft SQL Server](https://www.microsoft.com/en-us/sql-server)
* [PostgreSQL](https://www.postgresql.org/)

Why use a database? There are several reasons:

* Your data may already be in a database, so converting to another format is extra work.
* Database operations are highly optimized, so they typically take less time and memory than an equivalent operation in Python.
* Database operations can run on datasets that are too large to fit in memory. Doing this in Python requires special programming strategies.
* Many DBMSes provide built-in version control, multi-user access, and security checks.
* Databases can be updated in real time.

## Structured Query Language

_Structured query language_ (SQL) is a language designed for querying information in relational databases.

A free SQL tutorial is available [here](https://www.w3schools.com/sql/).

### Getting Connected

There are several ways to connect to a database and run SQL queries from Python:

* The built-in __sqlite3__ module, which only supports SQLite.
* The __sqlalchemy__ package, a unified interface for a variety of different SQL database formats (more than just SQLite). See the [tutorial](http://docs.sqlalchemy.org/en/latest/core/tutorial.html) for more details.

We'll use a SQLite database here, since SQLite is possibly [the most-used database engine in the world](https://sqlite.org/mostdeployed.html). SQLite's popularity is partly due to its reliability, easy setup, and broad range of features.

Let's connect to the suppliers database:

In [ ]:
import sqlite3 as sql

To connect to a database, use the module's `connect()` function. This is similar to opening a file; you should close the database when you're done using it.

To execute a SQL query, use the connection's `.execute()` method. This returns a _cursor_, which is a pointer to the results in the database (imagine a finger pointing at the results).

SQLite databases store metadata in a special table called `sqlite_master`. We can use `sqlite_master` to find out the names of the other tables in the database.

To get the results from the database, use one of the cursor's fetch methods. The `.fetchall()` method returns all rows in the result.

By default, `sqlite3` will return rows as tuples. If you'd rather have the rows as dictionaries indexed by column name, set the `.row_factory` attribute on the database connection.

Now the rows will behave like dictionaries:

Don't forget to close the database when you're done!

In [ ]:
# db.close()

We'll generally use the `pd.read_sql()` function in __pandas__ to run our SQL queries. 

The function takes a SQL query and an open database connection as arguments, so you still need to connect to the database first with `sqlite3` or `sqlalchemy`. The result of the query is returned as a data frame.

### `SELECT`

The `SELECT` command selects rows from a table. Most of your SQL queries will start with `SELECT`. The syntax is:

```sql
SELECT col1, col2, ... FROM my_table;
```

Here `col1`, `col2`, and so on are column names and `my_table` is a table name. You can select all columns with an asterisk  `*`.

SQL is not case-sensitive and ignores whitespace, but the convention is to write SQL keywords in uppercase and column/table names in lowercase. A semicolon `;` marks the end of a SQL query, but this is optional for many tools.

### `LIMIT`

The `SELECT` command can be extended with many other keywords.

The first of these is `LIMIT`, which limits the number of rows returned. `LIMIT` is the SQL equivalent of Pandas' `.head()` method.

### `DISTINCT`

The `DISTINCT` keyword limits rows to distinct results. `DISTINCT` is the SQL equivalent of Pandas' `.drop_duplicates()` method.

Keep in mind that `DISTINCT` applies to all of the selected columns, not just one column.

### `ORDER BY`

The `ORDER BY` keyword sorts the returned rows. `ORDER BY` is the SQL equivalent of Pandas' `.sort_values()` method.

Add the suffix `ASC` for an ascending sort (smallest to largest) and `DESC` for a descending sort (largest to smallest).

In SQLite, the default is ascending, but other other databases may differ.

### `WHERE`

`WHERE` puts conditions on the rows returned. `WHERE` is the SQL equivalent of subsetting.

You can use `=` to test equality. Other comparison operators, such as `>=`, are also available.

You can use `AND` and `OR` to combine conditions. You can also use parenthesis to indicate the order of operations.

You can use `IN` to check whether a value is in a collection of values.

SQL's `LIKE` keyword does simple pattern-matching language for strings. This is less powerful than regular expressions, but still useful.

* `%` matches zero or more of any character, similar to regex `.*`
* `_` matches any one character, similar to regex `.`

In other databases (but not SQLite):
* `[]` matches any one of the characters you put inside the brackects, identical to regex `[]`

The `BETWEEN` keyword is useful for selecting ranges.

### Operators

You can use arithmetic operators `+`, `-`, `*`, `\`, `%` on SQL columns to perform columnwise computations. These are the SQL equivalent of vectorized arithmetic.

### `AS`

You can rename a column with the `AS` keyword. This keyword is especially useful together with SQL arithmetic operators and functions.

### Functions & Aggregation

SQL has built-in functions, which vary from one DBMS to another. The SQL cheatsheet lists most of the functions supported by SQLite.

Most SQL functions aggregate data in a column, summarizing that column somehow.

### `GROUP BY`

The `GROUP BY` keyword groups rows before they are aggregated. `GROUP BY` is the SQL equivalent of Pandas' `.groupby()` method.

### `HAVING`

The `WHERE` keyword puts conditions on the rows returned _before computing any aggregate functions._ So use `WHERE` to remove rows before aggregation.

The `HAVING` keyword puts conditions on the rows returned _after computing any aggregate functions._ So use `HAVING` to remove rows after aggregation. `HAVING` is most useful for conditions on the result of an aggregate.

### Joins

A `JOIN` combines two tables using a column they have in common. `JOIN` is the SQL equivalent of Pandas' `.join()` method and `pd.merge()` function.

SQLite supports two kinds of joins:
* Left join (`LEFT JOIN`): Keep all rows from the left table, merging rows from the right.
* Inner join (`INNER JOIN`): Keep only rows where there is a match.

Other DBMSes sometimes support two more kinds of joins:
* Right join: Keep all rows from the right table, merging rows from the left.
* Full join: Keep all rows from both tables, merging where there are matches.

### Subqueries

You can write one or more `SELECT` queries within another:

### Database Operations: CRUD

So far we've only learned 1/4 of the basic operations most databases support. They are:

* Create (`CREATE`): Create a new table.
* Read (`SELECT`): Read some rows from a table.
* Update (`UPDATE`): Update rows in a table.
* Delete (`DROP`): Delete a table.

For data analysis, you'll usually only need to read data.

Once in a while, it can be useful to create tables to store complicated results. We have to go back to __sqlite3__ to run SQL commands other than `SELECT`.

## Other Databases

You can use the __sqlalchemy__ package to connect to many different kinds of databases (not only SQLite).

The `sqlalchemy.create_engine()` function creates an _engine_ to connect to a database. The `.raw_connection()` method on the returned engine opens a connection compatible with Pandas.

See the __sqlalchemy__ documentation for more info about [how to write a database URL](https://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls).

In [ ]:
import sqlalchemy